In [1]:
import os

In [2]:
%pwd

'/Users/cyber/MLProjectWineQuality/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/cyber/MLProjectWineQuality'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [35]:
from src.MLProject.constants import *
from src.MLProject.utils.common import read_yml, create_directories

In [36]:
class ConfigurationManager:
    
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yml(config_filepath)
        self.params = read_yml(params_filepath)
        self.schema = read_yml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [1]:
import os
import urllib.request as request
import zipfile
from MLProject import logger
from src.MLProject.utils.common import get_size

ModuleNotFoundError: No module named 'src'